# Step 1: Exchange pairs

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import time

In [212]:
df_matched = pd.read_stata('sample_matched_NY.dta')
df_unmatched = pd.read_stata('sample_unmatched_NY.dta')

'''
df_matched = pd.read_stata('test_sample_matched.dta')
df_unmatched = pd.read_stata('sample_unmatched.dta')
'''
def Exchange_pairs(df_matched, df_unmatched):
    # t1 = time.time()
    
    df_matched_column = df_matched.columns
    df_matched.columns = df_matched_column + '1m'
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id', 'lender_id1m')
    df1 = pd.merge(df_matched, df_unmatched, on = 'lender_id1m', how = 'inner')
    
    l = df1.columns[:-6].append([df1.columns[-6:] + '1um'])
    df1.columns = l
    df_matched.columns = df_matched_column
    df2 = pd.merge(df_matched, df1, left_on = 'loan_id', right_on = 'loan_id1um', how = 'inner') 
    
    ll = (df2.columns[:7]+'2m').append(df2.columns[7:])
    df2.columns = ll
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id1m', 'lender_id')
    df3 = pd.merge(df_unmatched, df2, left_on = ['lender_id','loan_id'], right_on = ['lender_id2m','loan_id1m'], how = 'inner')
    lll = (df3.columns[:7]+'2um').append(df3.columns[7:])
    df3.columns = lll
    
    df_keep = pd.DataFrame()
    for i in range(1, 6):
        name = "value" + str(i)
        df_keep[name] = df3["var"+str(i)+"1m"] + df3["var"+str(i)+"2m"] - df3["var"+str(i)+"1um"] - df3["var"+str(i)+"2um"]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_keep
df_keep = Exchange_pairs(df_matched, df_unmatched)
df_keep

,value1,value2,value3,value4,value5
0,1.0,-128.862122,95195504.0,0.000000,0.0
1,1.0,-123.937958,96773328.0,0.000000,0.0
2,0.0,94.940582,97036304.0,0.000000,0.0
3,0.0,-97.312332,23914.0,3.413793,0.0
4,0.0,-97.312332,0.0,0.413793,1.0
...,...,...,...,...,...
11222911,0.0,-163.896729,-27316.0,2.000000,-1.0
11222912,0.0,-163.896729,109264.0,2.600000,-1.0
11222913,0.0,-152.651489,-54632.0,3.000000,0.0
11222914,0.0,-31.092916,-109264.0,4.000000,-1.0


# Step 2: Define objective function 

Step 2: Define objective function -- (P15 equation 7 in Fox (2018) or P818 equation 4 in Schwert (2018) )
define a function that calculates the score of the matching model for given parameters
i.e. count how many inequalities are satisfied under given parameters 
objective function (score) =sum of 1(inequalities satisfied), where
$$Inequalities = beta \times value \geqslant 0$$
$\beta$ is the parameter vector
Value vector for a given data point (i.e. value 1-6 for a given data point)
Hint: apply (multiply) the parameter vector to the data matrix (transposed) and then count the non-negative values

The objective function $$Q(\beta) = \Sigma \mathbb{1}[X_{N\times 6}\beta_{6\times 1}]$$

# Step 3: Differential Evolution

In [64]:
# use differential_evolution in scipy.optimize to find the parameters than maximize the score calculated in step 2
# Reference
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html
# https://pablormier.github.io/2017/09/05/a-tutorial-on-differential-evolution-with-python/

In [18]:
df_test = df_keep.iloc[: 5, :]
df_test

,value1,value2,value3,value4,value5
0,1.0,-128.862122,95195504.0,0.000000,0.0
1,1.0,-123.937958,96773328.0,0.000000,0.0
2,0.0,94.940582,97036304.0,0.000000,0.0
3,0.0,-97.312332,23914.0,3.413793,0.0
4,0.0,-97.312332,0.0,0.413793,1.0


In [161]:
# 1. When input, make sure df contains all the values you need
# 2. specify df = X in def objectfunc(beta, df = df_test) before you run differential_evolution
def objectfunc(beta, df = df_keep):
    '''
    num = df.shape[1] # This is the num of Parameters( dimension of beta)
    obj = 0
    for i in range(0, num):
        obj = obj + np.array(df.iloc[:, i]) * beta[i]
        # print(obj)
    return -sum(obj >= 0)
    '''
    return -sum(df.dot(beta) >=0 )

In [162]:
# This is a test for debugging
beta = np.array([1, 2, 3, 4, 5])
obj = objectfunc(beta, df_keep)
obj

-5908089

In [163]:
# This is a test for debugging
beta = np.array([1, 2, 3, 4, 5])
obj = objectfunc(beta, df_test)
obj

-4

In [164]:
# specify bounds of every parameters
'''
num = df_test.shape[0]
num = df_test.shape[0]
bounds = [(-100, 100)] * num
'''
t1 = time.time()
bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)]
result = differential_evolution(objectfunc, bounds)
print(result)
t2 = time.time()
print("Running time: ", t2-t1)

     fun: -5989014.0
 message: 'Optimization terminated successfully.'
    nfev: 306
     nit: 3
 success: True
       x: array([  1.        ,  -4.47262905,  36.31679535, -41.54346339,
        88.40841153])
Running time:  364.816956281662


In [167]:
print("The bumber of inequalities satisfied is")
print(round(-result.fun))

The bumber of inequalities satisfied is
5989014


# Step 4: Use subsampling to construct confidence intervals

The idea is to select some random subsamples (100-200) that mimics the original sample and re-do the step1-3 and get another estimated parameter. Then calculate the 5% and 95% percentile of the 100 (200) parameters.
To mimic the original sample, we choose the subsample for fixed share for each state.

In [180]:
'''
# The random sampling ratio = 10% here
def Subsample(df, ratio):
    # t1 = time.time()
    df_sample = df.loc[np.random.choice(df.index, round(len(df) * ratio), replace = False)]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_sample
df_matched_sample = Subsample(df_matched, 0.1)
df_unmatched_sample = Subsample(df_unmatched, 0.1)
'''

'\n# The random sampling ratio = 10% here\ndef Subsample(df, ratio):\n    # t1 = time.time()\n    df_sample = df.loc[np.random.choice(df.index, round(len(df) * ratio), replace = False)]\n    # t2 = time.time()\n    # print("Running time: ", t2-t1)\n    return df_sample\ndf_matched_sample = Subsample(df_matched, 0.1)\ndf_unmatched_sample = Subsample(df_unmatched, 0.1)\n'

In [349]:
df_matched

,lender_id,loan_id,var1,var2,var3,var4,var5
0,814.0,19445.0,0.0,11.481559,2150.0,0.0,0.0
1,275.0,3498.0,1.0,169.902786,11088.0,0.0,0.0
2,3096.0,60530.0,1.0,7.044008,2823.0,0.0,0.0
3,3813.0,71810.0,0.0,1.056799,28242.0,0.0,0.0
4,60.0,456.0,0.0,3.011763,1764.0,0.0,0.0
...,...,...,...,...,...,...,...
6000,1085.0,23929.0,0.0,222.663376,38430.0,0.0,0.0
6001,1085.0,23938.0,0.0,222.663376,96075.0,0.0,0.0
6002,1795.0,36141.0,0.0,20.144510,5346.0,0.0,0.0
6003,1795.0,36138.0,0.0,17.873976,5346.0,0.0,0.0


In [351]:
# num is the # of random samples(or the # of simulations)
# ratio is the random sampling ratio, which is 10% here
def Simulation(num, ratio, df_matched, df_unmatched):
    df_result = pd.DataFrame()
    for i in range(0, num):
        # df_matched_sample = Subsample(df_matched, ratio)
        # df_unmatched_sample = Subsample(df_unmatched, ratio)
        
        loan_id = df_unmatched["loan_id"].unique()
        sample_loan_id = np.random.choice(loan_id, round(loan_id.shape[0] * ratio), replace = False)
        df_unmatched_sample = df_unmatched.loc[df_unmatched["loan_id"].isin(sample_loan_id)]
        df_matched_sample = df_matched.loc[df_matched["loan_id"].isin(sample_loan_id)]
       
        df_keep = Exchange_pairs(df_matched_sample, df_unmatched_sample)
        bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)] # fix beta_1 = 1
        result = differential_evolution(objectfunc, bounds)
        df_result = df_result.append(pd.Series(result.x), ignore_index = True)
        # print(i)
    print("The 5% quantile of parameters are")
    print(df_result.quantile(0.05))
    print("The 95% quantile of parameters are")
    print(df_result.quantile(0.95))
    return df_result


t1 = time.time()
df_result = Simulation(10, 0.1, df_matched, df_unmatched)
t2 = time.time()
print("Simulation time: ", t2-t1)
df_result

KeyboardInterrupt: 

In [177]:
print("The 5% quantile of parameters are")
print(df_result.quantile(0.05))
print("The 95% quantile of parameters are")
print(df_result.quantile(0.95))

The 5% quantile of parameters are
0     1.000000
1   -92.747828
2     0.054753
3   -31.420180
4   -82.814493
Name: 0.05, dtype: float64
The 95% quantile of parameters are
0     1.000000
1    -0.477537
2    65.198585
3    80.092220
4    97.542895
Name: 0.95, dtype: float64


# Step 5: Model Fitness

## 1) Number of inequalities satisfied: count how many inequalities are satisfied under the optimized parameter choice.

In [178]:
def Fox_func(path1, path2, num, ratio):
    df_matched = pd.read_stata(path1)
    df_unmatched = pd.read_stata(path2)
    df_keep = Exchange_pairs(df_matched, df_unmatched)
    def objectfunc(beta, df = df_keep):
        return -sum(df.dot(beta) >=0 )
    t1 = time.time()
    bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)]
    result = differential_evolution(objectfunc, bounds)
    print(result)
    t2 = time.time()
    print("Differential Evolution time: ", t2-t1)
    print("The bumber of inequalities satisfied is")
    print(round(-result.fun))
    Simulation(num, ratio, df_matched, df_unmatched)

In [ ]:
Fox_func("sample_matched_NY_demean1.dta", "sample_unmatched_NY_demean1.dta", 100, 0.1)

     fun: -6162188.0
 message: 'Optimization terminated successfully.'
    nfev: 906
     nit: 11
 success: True
       x: array([  1.        , -42.00088777,   0.11136153, -52.85150716,
       -63.17221436])
Differential Evolution time:  1093.1349799633026
The bumber of inequalities satisfied is
6162188
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [ ]:
Fox_func("sample_matched_NY_org.dta", "sample_unmatched_NY_org.dta", 100, 0.1)

In [ ]:
Fox_func("sample_matched_NY_std.dta", "sample_unmatched_NY_org.std", 100, 0.1)

## 2) In sample prediction: assign firms to lenders according to the estimated value function and count how many of the observed (P820, footnote 51 – Schwert (2018)) Target Output – Table IX X in Schwert (2018)

- Calculate the quota for each lender.
- Using the estimated value function, matching values are calculated for all pairs in matched sample and unmatched sample.
- For every lender, rank the pairs by matching values in descending order and pick the top "quota" pairs.
- Calculate how many pairs are in matched sample

In [314]:
values, counts = np.unique(df_matched["lender_id"], return_counts = True)
df_quota = pd.DataFrame(counts, index = values, columns = ["quota"])
df_quota

,quota
14.0,1
49.0,13
57.0,1
60.0,5
67.0,28
...,...
5001.0,8
5007.0,34
5009.0,1
5084.0,1


In [315]:
df_total = df_matched.append(df_unmatched)
df_total

,lender_id,loan_id,var1,var2,var3,var4,var5
0,814.0,19445.0,0.0,11.481559,2150.0,0.0,0.0
1,275.0,3498.0,1.0,169.902786,11088.0,0.0,0.0
2,3096.0,60530.0,1.0,7.044008,2823.0,0.0,0.0
3,3813.0,71810.0,0.0,1.056799,28242.0,0.0,0.0
4,60.0,456.0,0.0,3.011763,1764.0,0.0,0.0
...,...,...,...,...,...,...,...
430180,3278.0,36138.0,0.0,174.711655,1337778.0,0.0,0.0
430181,697.0,19525.0,0.0,1279.236694,772416.0,0.0,0.0
430182,697.0,36138.0,0.0,1279.236694,2317248.0,0.0,0.0
430183,3923.0,36138.0,0.0,0.000000,515700.0,4.5,0.0


In [316]:
beta = result.x
beta

array([  1.        ,  -4.47262905,  36.31679535, -41.54346339,
        88.40841153])

In [317]:
df_total["matching_value"] = df_total.iloc[:,2:].dot(beta)
df_total

,lender_id,loan_id,var1,var2,var3,var4,var5,matching_value
0,814.0,19445.0,0.0,11.481559,2150.0,0.0,0.0,7.802976e+04
1,275.0,3498.0,1.0,169.902786,11088.0,0.0,0.0,4.019217e+05
2,3096.0,60530.0,1.0,7.044008,2823.0,0.0,0.0,1.024918e+05
3,3813.0,71810.0,0.0,1.056799,28242.0,0.0,0.0,1.025654e+06
4,60.0,456.0,0.0,3.011763,1764.0,0.0,0.0,6.404936e+04
...,...,...,...,...,...,...,...,...
430180,3278.0,36138.0,0.0,174.711655,1337778.0,0.0,0.0,4.858303e+07
430181,697.0,19525.0,0.0,1279.236694,772416.0,0.0,0.0,2.804595e+07
430182,697.0,36138.0,0.0,1279.236694,2317248.0,0.0,0.0,8.414930e+07
430183,3923.0,36138.0,0.0,0.000000,515700.0,4.5,0.0,1.872838e+07


In [348]:
s = 0
for i in df_quota.index:
    quota = int(df_quota.loc[i])
    id_sorted = df_total[df_total["lender_id"] == i].sort_values(by = "matching_value")[0:quota]["loan_id"]
    s = s + sum(df_matched[df_matched["lender_id"] == i]["loan_id"].isin(id_sorted))
print(s/df_matched.shape[0])

0.07610324729392173
